# Detecting the fiducial particles in LM images

The fiducial particles appear as bright spots in the LM images. In this notebook, we **detect fiducial particles** in **LM images** using Big-FISH - a python package originally created for analysis of smFISH images. It includes various methods, such as spot detection which we are using in this notebook. The main steps of this algorithm then are:
- **1. Spot detection** - Detection of fiducial particles using spot detection methods from Big-FISH package
- **2. Dense region decomposition** - Recognizing larger and brighter (dense) spots as clusters and estimating the number of individual fiducial particles building these clusters
- **3. Save results** - Saving the positions of all detected fiducial particles and the positions of fiducial clusters into files

Load the necessary python libraries:

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import os
import cv2
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from utils import plot_image, list_to_dataframe, dataframe_to_xml_, dataframe_to_pointcloud
import open3d as o3d
from probreg import cpd
from pathlib import Path
import bigfish
import bigfish.stack as stack
import bigfish.detection as detection
import bigfish.multistack as multistack
import bigfish.plot as plot
print("Big-FISH version: {0}".format(bigfish.__version__))

Set the path to the input EM and LM images and load them

In [ ]:
input_folder = 'E:/DATA/AI4Life_Pr26/20240805_Trial_data_fiducial_particles/240723_JB294_CLEM-AI4life_sample1/pos1'
test_folder = '//vironova.com/root/Users/kristinal/Documents/1Test'

EM_image_path = os.path.join(input_folder, "240726_JB295_HEK293_CLEM_LAMP1-488_Particles-555_grid4_pos1_bin4_EM.tif")
LM_image_path = os.path.join(input_folder, "C2-experiment_ai4life.tif")

print(Path(EM_image_path).exists())
print(Path(LM_image_path).exists())

output_folder = Path(os.path.join(input_folder,"output"))
output_folder.mkdir(exist_ok=True)

EMimage = stack.read_image(EM_image_path)
LMimage = stack.read_image(LM_image_path)

print("EMimage")
print("\r shape: {0}".format(EMimage.shape))
print("\r dtype: {0}".format(EMimage.dtype))

print("LMimage:")
print("\r shape: {0}".format(LMimage.shape))
print("\r dtype: {0}".format(LMimage.dtype))

Check the size of the bigger EM image and rescale the LM image accordingly:

In [ ]:
resized_LMimage = stack.resize_image(LMimage, EMimage.shape, method='bilinear')

print("resized_LMimage:")
print("\r shape: {0}".format(resized_LMimage.shape))
print("\r dtype: {0}".format(resized_LMimage.dtype))

scale_x = EMimage.shape[0]/LMimage.shape[0]
scale_y = EMimage.shape[1]/LMimage.shape[1]

print(scale_x)
print(scale_y)

In [ ]:
#overlaid_image = stack.maximum_projection(np.stack([resized_LMimage, EMimage]))
#plot.plot_images(overlaid_image, contrast=True, framesize=(5, 5))

fig = plt.figure(figsize=(40, 40))
plt.imshow(resized_LMimage, cmap='gray', alpha=0.5)
plt.imshow(EMimage, cmap='gray', alpha=0.5)
plt.show()

## 1. Spot detection

We assume **spot is a local maximum** in the LM image. Three steps are required to detect them:
- Filter the LM image to enhance the signal-to-noise ratio and denoise the image (`bigfish.stack.log_filter`).
- Detect the local maximum in the filtered image (`bigfish.detection.local_maximum_detection`).
- Remove the local maximum under a fixed threshold (`bigfish.detection.spots_thresholding`). To be robust, the thresholding should be applied on the filtered image. Thus, the threshold is set relatively to the filtered image values.
- If necessary, the optimal threshold can be estimated with `bigfish.detection.automated_threshold_setting` (applied on a filtered image).

All these steps are summarized in `bigfish.detection.detect_spots` that return the 2D or 3D coordinates of the detected spots.

In [ ]:
vox_size = 110  # voxel size in nanometers
spot_rad = 100 

spots, threshold = detection.detect_spots(
    images=LMimage, 
    return_threshold=True, 
    voxel_size=(vox_size, vox_size),  # in nanometer (one value per dimension zyx)
    spot_radius=(spot_rad, spot_rad))  # in nanometer (one value per dimension zyx)

# From Fiji file:
# Width:  29.2602 microns (266)
# Height:  71.2805 microns (648)
# Size:  1010K
# Resolution:  9.0908 pixels per micron
# Voxel size: 0.1100x0.1100x1 micron^3

print("detected spots")
print("\r shape: {0}".format(spots.shape))
print("\r dtype: {0}".format(spots.dtype))
print("\r threshold: {0}".format(threshold))

Given the **voxel size** and the expected **spot radius** (in nanometer), the function `bigfish.detection.detect_spots` automatically estimates a **kernel size** for the LoG filtering and a **minimal distance** between two spots we want to be able to detect separately. It is still possible to set these parameters explicitly in order to fine-tune the detection. Internally, we approximate them as the spot radius in pixel with the function `bigfish.detection.get_object_radius_pixel`.

In [ ]:
spot_radius_px = detection.get_object_radius_pixel(
    voxel_size_nm=(vox_size, vox_size), 
    object_radius_nm=(spot_rad, spot_rad), 
    ndim=2)
print("spot radius (z axis): {:0.3f} pixels".format(spot_radius_px[0]))
print("spot radius (yx plan): {:0.3f} pixels".format(spot_radius_px[-1]))

In [ ]:
spots, threshold = detection.detect_spots(
    images=LMimage, 
    return_threshold=True, 
    log_kernel_size=(1.000, 1.000),
    minimum_distance=(1.000, 1.000))
print("detected spots")
print("\r shape: {0}".format(spots.shape))
print("\r dtype: {0}".format(spots.dtype))
print("\r threshold: {0}".format(threshold))

__Note:__ What we call spot radius in this notebook can be understood as its **Point Spread Function (PSF)**. For simplicity sake, this PSF is modelled as a 2D or 3D gaussian.

The previous steps can be computed separately.

In [ ]:
# spot radius
spot_radius_px = detection.get_object_radius_pixel(
    voxel_size_nm=(vox_size, vox_size), 
    object_radius_nm=(spot_rad, spot_rad), 
    ndim=2)

# LoG filter
rna_log = stack.log_filter(LMimage, sigma=spot_radius_px)

# local maximum detection
mask = detection.local_maximum_detection(rna_log, min_distance=spot_radius_px)

# thresholding
threshold = detection.automated_threshold_setting(rna_log, mask)
threshold = 1500
spots, _ = detection.spots_thresholding(rna_log, mask, threshold)
print("detected spots")
print("\r shape: {0}".format(spots.shape))
print("\r dtype: {0}".format(spots.dtype))
print("\r threshold: {0}".format(threshold))

In [ ]:
plot.plot_detection(LMimage, spots, contrast=True)

The automated spot detection method tries to find the optimal threshold to discriminate actual spots from noisy blobs. If we plot the number of the spots detected as a function of threshold level we observe an **elbow curve**. The selected threshold is the one located in the breaking point of the curve. This curve can be plotted with `bigfish.plot.plot_elbow`.

In [ ]:
plot.plot_elbow(
    images=LMimage, 
    voxel_size=(vox_size, vox_size), 
    spot_radius=(spot_rad, spot_rad))

## 2. Dense region decomposition 

The detection of local maximum is not able to detect individual fiducial particles clustered in a dense and bright region. We try to **decompose these regions by simulating as many spots as possible until we match the original region intensity**. Our current steps are:
- Denoise the LM image by estimating then removing its background (`bigfish.stack.remove_background_gaussian`).
- Build a reference median spot from the individual predetected spots (`bigfish.detection.build_reference_spot`).
- Fit a gaussian signal on the reference spot (`bigfish.detection.modelize_spot`).
- Detect the candidate dense regions in the denoised image - large regions brighter than the reference spot (`bigfish.detection.get_dense_region`).
- Use the fitted gaussian signal to fill as many spots in the candidate regions as possible (`bigfish.detection.simulate_gaussian_mixture`).

All these steps are summarized in `bigfish.detection.decompose_dense` that return the 2D or 3D coordinates of the detected spots outside and inside a decomposed region, additional information about the regions themself and an image of the reference spot estimated.

In [ ]:
spots_post_decomposition, dense_regions, reference_spot = detection.decompose_dense(
    image=LMimage, 
    spots=spots, 
    voxel_size=(vox_size, vox_size), 
    spot_radius=(spot_rad, spot_rad), 
    alpha=0.3,  # alpha impacts the number of spots per candidate region
    beta=1,  # beta impacts the number of candidate regions to decompose
    gamma=5)  # gamma the filtering step to denoise the image
print("detected spots before decomposition")
print("\r shape: {0}".format(spots.shape))
print("\r dtype: {0}".format(spots.dtype), "\n")
print("detected spots after decomposition")
print("\r shape: {0}".format(spots_post_decomposition.shape))
print("\r dtype: {0}".format(spots_post_decomposition.dtype))

Alternatively, all the previous steps can be computed separately.

In [ ]:
# gaussian kernel
kernel_size = detection.get_object_radius_pixel(
    voxel_size_nm=(vox_size, vox_size), 
    object_radius_nm=(spot_rad, spot_rad), 
    ndim=2)
large_kernel_size = tuple([kernel_size_ * 5 for kernel_size_ in kernel_size])

# denoising
rna_denoised = stack.remove_background_gaussian(LMimage, sigma=large_kernel_size)

# reference spot
reference_spot = detection.build_reference_spot(
    image=rna_denoised,
    spots=spots,
    voxel_size=(vox_size, vox_size), 
    spot_radius=(spot_rad, spot_rad),
    alpha=0.7)

# fit a gaussian function on the reference spot
sigma_yx, amplitude, background = detection.modelize_spot(
    reference_spot=reference_spot, 
    voxel_size=(vox_size, vox_size), 
    spot_radius=(spot_rad, spot_rad))

# detect dense regions
regions_to_decompose, spots_out_regions, region_size = detection.get_dense_region(
    image=rna_denoised, 
    spots=spots,
    voxel_size=(vox_size, vox_size),
    spot_radius=(spot_rad, spot_rad),
    beta=1)

# precompute gaussian function values
max_grid = max(200, region_size + 1)
precomputed_gaussian = detection.precompute_erf(
    ndim=2,
    voxel_size=(vox_size, vox_size),
    sigma=(sigma_yx, sigma_yx),
    max_grid=max_grid)

# simulate gaussian mixtures
spots_in_regions, _ = detection.simulate_gaussian_mixture(
    image=rna_denoised,
    candidate_regions=regions_to_decompose,
    voxel_size=(vox_size, vox_size),
    sigma=(sigma_yx, sigma_yx),
    amplitude=amplitude,
    background=background,
    precomputed_gaussian=precomputed_gaussian)

spots_post_decomposition = np.concatenate((spots_out_regions, spots_in_regions[:, :2]), axis=0)
print("detected spots before decomposition")
print("\r shape: {0}".format(spots.shape))
print("\r dtype: {0}".format(spots.dtype), "\n")
print("detected spots after decomposition")
print("\r shape: {0}".format(spots_post_decomposition.shape))
print("\r dtype: {0}".format(spots_post_decomposition.dtype))

In [ ]:
plot.plot_detection(LMimage, spots_post_decomposition, contrast=True)

In [ ]:
#print(spots_post_decomposition)
# Comverting list of tuples into nparray
spots_inv = [[int(x*scale_y), int(y*scale_x)] for y, x in (spots_post_decomposition)]
spots_inv = [[int(x), int(y)] for y, x in (spots_post_decomposition)]
#loc = np.hstack((spots_inv, np.zeros((len(spots_inv), 1))))
print(spots_inv)

In [ ]:
spots_post_decomposition

In [ ]:
threshold

## 3. Save results

In [ ]:
# Save all detected fiducial particles into a Pandas dataframe

source_all_df = list_to_dataframe(spots_inv) #str(output_folder/"source_all_df.csv")

# source = dataframe_to_nparray(source_all_df)
dataframe_to_xml_(source_all_df,str(output_folder/"source_all.xml"))                      # str(output_folder/"source.xml")
source_pcd = dataframe_to_pointcloud(source_all_df, str(output_folder/"source_all.ply"))  # "str(output_folder/source.ply)"

print(source_all_df)

## Clusters detection

Two spots are considered connected if they localized within a specific radius (in nanometer). Above a minimum number of connected spots, a cluster can be defined. This detection can be computed with `bigfish.detection.detect_clusters`.

In [ ]:
spots_post_clustering, clusters = detection.detect_clusters(
    spots=np.asarray(spots_post_decomposition), 
    voxel_size=(110, 110), 
    radius=110, 
    nb_min_spots=3)
print("detected spots after clustering")
print("\r shape: {0}".format(spots_post_clustering.shape))
print("\r dtype: {0}".format(spots_post_clustering.dtype), "\n")
print("detected clusters")
print("\r shape: {0}".format(clusters.shape))
print("\r dtype: {0}".format(clusters.dtype))

In [ ]:
# plot
#plot.plot_detection(LMimage, spots_post_clustering, contrast=True)
plot.plot_detection(LMimage, 
                    spots=[spots_post_decomposition, clusters[:, :2]], 
                    shape=["circle", "circle"], 
                    radius=[3, 6], 
                    color=["red", "green"],
                    linewidth=[1, 2], 
                    fill=[False, False], 
                    contrast=True)

## Additional detection methods

We crop a part of our smFISH channel to better visualize the results of the next methods. Especially, we look at a region without too many clustered spots because it could negativally impact **subpixel fitting** and **colocalized spots detection**. 

In [ ]:
# crop
crop = LMimage[ 520:630, 150:260]
crop_mip = LMimage[520:630, 150:260]
print("smfish channel (crop)")
print("\r shape: {0}".format(crop.shape))
print("\r dtype: {0}".format(crop.dtype), "\n")

# plot
plot.plot_images(crop, contrast=True, framesize=(5, 5))

### Subpixel fitting

If you analyze few isolated spots, it is possible to refine spots detections at the subpixel level with `bigfish.detection.fit_subpixel` (in 2D and 3D).

In [ ]:
# pixel fitting
spots_crop = detection.detect_spots(
    images=crop,
    threshold=1500.0,  # previous threshold automatically find in the full image
    voxel_size=(vox_size, vox_size),
    spot_radius=(spot_rad, spot_rad))
print("spots (pixel fitting)")
print("\r shape: {0}".format(spots_crop.shape))
print("\r dtype: {0}".format(spots_crop.dtype), "\n")

# subpixel fitting
spots_subpixel_crop = detection.fit_subpixel(
    image=crop, 
    spots=spots_crop, 
    voxel_size=(vox_size, vox_size), 
    spot_radius=(spot_rad, spot_rad))
print("spots (subpixel fitting)")
print("\r shape: {0}".format(spots_subpixel_crop.shape))
print("\r dtype: {0}".format(spots_subpixel_crop.dtype))

In [ ]:
# plot
plot.plot_detection(
    crop_mip, 
    spots=[spots_crop, spots_subpixel_crop], 
    radius=2, 
    color=["red", "blue"],
    title="Red = pixel fitting | Blue = subpixel fitting",
    linewidth=2, contrast=True, framesize=(10, 5))

### Colocalization

To detect colocalized spots (over different channels), the function `bigfish.multistack.detect_spots_colocalization` is available. Here, as an example, we detect colocalized spots between the ones detected with a pixel accuracy and the ones with subpixel accuracy. Like the regular spot detection, if a threshold is not provided to discriminate colocalized spots from distant ones, the function tries to set one, automatically.

In [ ]:
(spots_1_colocalized, spots_2_colocalized, 
 distances, 
 indices_1, indices_2, 
 threshold) = multistack.detect_spots_colocalization(
    spots_1=spots_crop, 
    spots_2=spots_subpixel_crop,
    voxel_size=(110,110),
    return_indices=True,
    return_threshold=True)
print("colocalized spots")
print("\r shape 1: {0}".format(spots_1_colocalized.shape))
print("\r shape 2: {0}".format(spots_2_colocalized.shape))
print("\r distances: {0}".format(distances.shape))
print("\r indices 1: {0}".format(indices_1.shape))
print("\r indices 2: {0}".format(indices_2.shape))
print("\r threshold: {0:0.2f} nm".format(threshold))

In [ ]:
# plot
plot.plot_detection(
    crop_mip, 
    spots=[spots_1_colocalized, spots_2_colocalized], 
    radius=2, 
    color=["red", "blue"],
    title="Red = pixel fitting | Blue = subpixel fitting",
    linewidth=2, contrast=True, framesize=(10, 5))

Using the same logic than `bigfish.plot.plot_elbow`, the automated colocalized spot detection method tries to find the optimal threshold to discriminate colocalized spots from distant ones. One can use `bigfish.plot.plot_elbow_colocalized` to visualize the impact of the threshold on the colocalization.

In [ ]:
plot.plot_elbow_colocalized(
    spots_1=spots_crop, 
    spots_2=spots_subpixel_crop, 
    voxel_size=(110,110))

## Save results

Spots and foci coordinates can be saved in **npy files** (numpy dedicated format) or **csv files** using functions `bigfish.stack.save_array` and `bigfish.stack.save_data_to_csv` respectively.

In [ ]:
# save in npy files
path = os.path.join(path_output, "spots.npy")
stack.save_array(spots_post_clustering, path)
path = os.path.join(path_output, "clusters.npy")
stack.save_array(clusters, path)

# save in csv files
path = os.path.join(path_output, "spots.csv")
stack.save_data_to_csv(spots_post_clustering, path)
path = os.path.join(path_output, "clusters.csv")
stack.save_data_to_csv(clusters, path)

## Detection in 2D or 3D

Based on the number of dimensions of the provided image, a 2D or 3D detection is performed and corresponding coordinates are returned. Parameters `voxel_size` and `spot_radius` should be adapted to the number of dimensions.

In [ ]:
spots, threshold = detection.detect_spots(
    images=LMimage, 
    return_threshold=True, 
    voxel_size=(103, 103),  # in nanometer (one value per dimension yx)
    spot_radius=(150, 150))  # in nanometer (one value per dimension yx)
print("detected spots")
print("\r shape: {0}".format(spots.shape))
print("\r dtype: {0}".format(spots.dtype))
print("\r threshold: {0}".format(threshold))

In [ ]:
spots_post_decomposition, dense_regions, reference_spot = detection.decompose_dense(
    image=LMimage, 
    spots=spots, 
    voxel_size=(vox_size,vox_size), 
    spot_radius=(spot_rad,spot_rad), 
    alpha=0.3,  # alpha impacts the number of spots per candidate region
    beta=1,  # beta impacts the number of candidate regions to decompose
    gamma=5)  # gamma the filtering step to denoise the image
print("detected spots before decomposition")
print("\r shape: {0}".format(spots.shape))
print("\r dtype: {0}".format(spots.dtype), "\n")
print("detected spots after decomposition")
print("\r shape: {0}".format(spots_post_decomposition.shape))
print("\r dtype: {0}".format(spots_post_decomposition.dtype))

In [ ]:
spots_post_clustering, clusters = detection.detect_clusters(
    spots=spots_post_decomposition, 
    voxel_size=(103, 103), 
    radius=350, 
    nb_min_spots=4)
print("detected spots after clustering")
print("\r shape: {0}".format(spots_post_clustering.shape))
print("\r dtype: {0}".format(spots_post_clustering.dtype), "\n")
print("detected clusters")
print("\r shape: {0}".format(clusters.shape))
print("\r dtype: {0}".format(clusters.dtype))

In [ ]:
# plot
plot.plot_detection(LMimage, 
                    spots=[spots_post_decomposition, clusters[:, :2]], 
                    shape=["circle", "square"], 
                    radius=[3, 6], 
                    color=["red", "blue"],
                    linewidth=[1, 2], 
                    fill=[False, True], 
                    contrast=True)